In [1]:
from datetime import datetime
import pytz
from requests_html import HTMLSession
import mysql.connector
s = HTMLSession()

In [2]:
def get_local_datetime(query):
    timezone_bt = pytz.timezone('Europe/London')
    timezone_cet = pytz.timezone('Europe/Paris')
    timezone_eest = pytz.timezone('Europe/Bucharest')
    if query == 'london':
        local_datetime = str(datetime.now(timezone_bt))[0:19]
    elif query == 'nice':
        local_datetime = str(datetime.now(timezone_cet))[0:19]
    elif query == 'rome':
        local_datetime = str(datetime.now(timezone_cet))[0:19]
    elif query == 'mykonos':
        local_datetime = str(datetime.now(timezone_eest))[0:19]
    elif query == 'cluj-napoca':
        local_datetime = str(datetime.now(timezone_eest))[0:19]
    elif query == 'copenhagen':
        local_datetime = str(datetime.now(timezone_cet))[0:19]
    elif query == 'stockholm':
        local_datetime = str(datetime.now(timezone_cet))[0:19]
    elif query == 'oslo':
        local_datetime = str(datetime.now(timezone_cet))[0:19]
    return local_datetime

In [3]:
def get_location_id(query):
    if query == 'london':
        loc_id = 1
    elif query == 'nice':
        loc_id = 2
    elif query == 'rome':
        loc_id = 3
    elif query == 'mykonos':
        loc_id = 4
    elif query == 'cluj-napoca':
        loc_id = 5
    elif query == 'copenhagen':
        loc_id = 6
    elif query == 'stockholm':
        loc_id = 7
    elif query == 'oslo':
        loc_id = 8
    return loc_id

In [4]:
def get_weather_data(query):
    loc_id = get_location_id(query)
    local_time = get_local_datetime(query)
    url = f'https://www.google.com/search?q=weather+{query}'
    r = s.get(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36',  "Accept-Language": "en-UK,en;q=0.9,ar;q=0.8"})
    temp = r.html.find('span#wob_tm', first = True).text
    unit = r.html.find('div.vk_bk.wob-unit span.wob_t', first= True).text
    descr = r.html.find('div.VQF4g', first = True).find('span#wob_dc', first = True).text
    prec = r.html.find('span#wob_pp', first = True).text
    humi = r.html.find('span#wob_hm', first = True).text
    wind = r.html.find('span#wob_ws', first = True).text
    return loc_id, local_time, temp, unit, descr, prec, humi, wind

In [5]:
mydb = mysql.connector.connect(
    host='weather-database-instance-1.cftovd3de2b5.eu-west-1.rds.amazonaws.com',
    user='admin', 
    passwd='rootroot', 
    database='weather')

In [6]:
mycursor = mydb.cursor()

In [7]:
sql = "INSERT INTO weather_data (location_id, local_time, temperature, measureament_unit, description, precipitations, humidity, wind) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
queries = ['london', 'nice', 'rome', 'mykonos', 'cluj-napoca', 'copenhagen', 'stockholm', 'oslo']
for query in queries:
    loc_id, local_time, temp, unit, descr, prec, humi, wind = get_weather_data(query)
    val = (loc_id, local_time, temp, unit, descr, prec, humi, wind)
    mycursor.execute(sql, val)
    mydb.commit()